In [2]:
import pandas as pd
import numpy as np
import numpy as np,pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
import joblib
from sklearn.impute import SimpleImputer



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


PREPROCESSING THE DATA

In [4]:
first_df=pd.read_csv('/content/drive/MyDrive/AI Projects/players_21.csv')

In [5]:
first_df

sofifa_id                                         player_url  \
0         158023  https://sofifa.com/player/158023/lionel-messi/...   
1          20801  https://sofifa.com/player/20801/c-ronaldo-dos-...   
2         188545  https://sofifa.com/player/188545/robert-lewand...   
3         190871  https://sofifa.com/player/190871/neymar-da-sil...   
4         192985  https://sofifa.com/player/192985/kevin-de-bruy...   
...          ...                                                ...   
18939     257710  https://sofifa.com/player/257710/mengxuan-zhan...   
18940     257933  https://sofifa.com/player/257933/wenzhuo-huang...   
18941     257936   https://sofifa.com/player/257936/yue-song/210002   
18942     258736  https://sofifa.com/player/258736/vani-da-silva...   
18943     258760  https://sofifa.com/player/258760/ben-hough/210002   

              short_name                            long_name  \
0               L. Messi       Lionel Andrés Messi Cuccittini   
1      Cristiano Ronaldo  Cristiano Ronaldo dos Santos Aveiro   
2         R. Lewandowski                   Robert Lewandowski   
3              Neymar Jr        Neymar da Silva Santos Júnior   
4           K. De Bruyne                      Kevin De Bruyne   
...                  ...                                  ...   
18939     Zhang Mengxuan                                  张梦炫   
18940      Huang Wenzhou                                  黄文卓   
18941           Song Yue                                   宋岳   
18942        V. Da Silva     Ivanilson Loforte Tique Da Silva   
18943           B. Hough                            Ben Hough   

      player_positions  overall  potential    value_eur  wage_eur  age  ...  \
0           RW, ST, CF       93         93  103500000.0  560000.0   33  ...   
1               ST, LW       92         92   63000000.0  220000.0   35  ...   
2                   ST       91         91  111000000.0  240000.0   31  ...   
3              LW, CAM       91         91  132000000.0  270000.0   28  ...   
4              CAM, CM       91         91  129000000.0  370000.0   29  ...   
...                ...      ...        ...          ...       ...  ...  ...   
18939               CB       47         52      70000.0    1000.0   21  ...   
18940               CM       47         53      70000.0    1000.0   21  ...   
18941               CM       47         47      45000.0    2000.0   28  ...   
18942               ST       47         67     130000.0     500.0   17  ...   
18943               CM       47         67     130000.0     500.0   17  ...   

        lcb    cb   rcb    rb    gk  \
0      52+3  52+3  52+3  62+3  19+3   
1      54+3  54+3  54+3  61+3  20+3   
2      60+3  60+3  60+3  61+3  19+3   
3      49+3  49+3  49+3  62+3  20+3   
4      69+3  69+3  69+3  75+3  21+3   
...     ...   ...   ...   ...   ...   
18939  47+2  47+2  47+2  45+2  15+2   
18940  46+2  46+2  46+2  47+2  15+2   
18941  46+1  46+1  46+1    47  11+2   
18942  26+2  26+2  26+2  33+2  14+2   
18943  41+2  41+2  41+2  43+2  14+2   

                                         player_face_url  \
0      https://cdn.sofifa.net/players/158/023/21_120.png   
1      https://cdn.sofifa.net/players/020/801/21_120.png   
2      https://cdn.sofifa.net/players/188/545/21_120.png   
3      https://cdn.sofifa.net/players/190/871/21_120.png   
4      https://cdn.sofifa.net/players/192/985/21_120.png   
...                                                  ...   
18939  https://cdn.sofifa.net/players/257/710/21_120.png   
18940  https://cdn.sofifa.net/players/257/933/21_120.png   
18941  https://cdn.sofifa.net/players/257/936/21_120.png   
18942  https://cdn.sofifa.net/players/258/736/21_120.png   
18943  https://cdn.sofifa.net/players/258/760/21_120.png   

                                    club_logo_url  \
0         https://cdn.sofifa.net/teams/241/60.png   
1          https://cdn.sofifa.net/teams/45/60.png   
2          https://cdn.sofifa.net/teams/21/60.png   
3          https://cdn

In [6]:
first_df.isnull().sum()

sofifa_id               0
player_url              0
short_name              0
long_name               0
player_positions        0
                    ...  
player_face_url         0
club_logo_url         225
club_flag_url         225
nation_logo_url     17817
nation_flag_url         0
Length: 110, dtype: int64

In [7]:
label_encoder = LabelEncoder()
first_df['player_positions'] = label_encoder.fit_transform(first_df['player_positions'])

In [8]:
first_df['player_positions'].head

<bound method NDFrame.head of 0        539
1        587
2        560
3        331
4         10
        ... 
18939     55
18940    149
18941    149
18942    560
18943    149
Name: player_positions, Length: 18944, dtype: int64>

In [9]:
correlation_with_overall = first_df.corrwith(first_df['overall'])
sorted_correlations = correlation_with_overall.sort_values(ascending=False)
sorted_correlations[:35]

<ipython-input-9-9fba18009bb6>:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_with_overall = first_df.corrwith(first_df['overall'])


overall                        1.000000
movement_reactions             0.867234
passing                        0.711521
mentality_composure            0.705252
dribbling                      0.641097
potential                      0.636366
release_clause_eur             0.626277
wage_eur                       0.592452
value_eur                      0.558493
power_shot_power               0.558372
physic                         0.530386
mentality_vision               0.509087
attacking_short_passing        0.502191
goalkeeping_speed              0.497350
shooting                       0.488315
skill_long_passing             0.487147
age                            0.468197
skill_ball_control             0.449372
international_reputation       0.440987
skill_curve                    0.420495
attacking_crossing             0.410530
power_long_shots               0.407525
mentality_aggression           0.401920
skill_fk_accuracy              0.385617
power_stamina                  0.381869


In [10]:
best_features=first_df[['overall', 'potential','mentality_composure', 'mentality_vision','passing', 'dribbling', 'release_clause_eur', 'wage_eur', 'value_eur',
'power_shot_power', 'shooting', 'power_long_shots','physic','age','international_reputation','attacking_short_passing', 'skill_long_passing', 'skill_ball_control', 'skill_fk_accuracy',
'skill_moves', 'skill_dribbling', 'skill_curve', 'attacking_crossing', 'attacking_volleys', 'attacking_heading_accuracy', 'attacking_finishing',
'movement_reactions','power_strength', 'power_stamina','mentality_positioning', 'mentality_penalties','defending', 'defending_marking_awareness']]

In [11]:
df=pd.DataFrame(best_features)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  int64  
 1   potential                    18944 non-null  int64  
 2   mentality_composure          18944 non-null  int64  
 3   mentality_vision             18944 non-null  int64  
 4   passing                      16861 non-null  float64
 5   dribbling                    16861 non-null  float64
 6   release_clause_eur           17949 non-null  float64
 7   wage_eur                     18719 non-null  float64
 8   value_eur                    18707 non-null  float64
 9   power_shot_power             18944 non-null  int64  
 10  shooting                     16861 non-null  float64
 11  power_long_shots             18944 non-null  int64  
 12  physic                       16861 non-null  float64
 13  age             

In [13]:
#Replacing all NaN values with mean
imp = SimpleImputer(strategy='mean')
imp.fit(df)
imputed_data=imp.fit_transform(df)
df=pd.DataFrame(imputed_data, columns=df.columns)

# Replace 93 overall with 92 if they appear only once
unique_93_overall = df['overall'].value_counts()[93]
if unique_93_overall == 1:
    df['overall'].replace(93, 92, inplace=True)



In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  float64
 1   potential                    18944 non-null  float64
 2   mentality_composure          18944 non-null  float64
 3   mentality_vision             18944 non-null  float64
 4   passing                      18944 non-null  float64
 5   dribbling                    18944 non-null  float64
 6   release_clause_eur           18944 non-null  float64
 7   wage_eur                     18944 non-null  float64
 8   value_eur                    18944 non-null  float64
 9   power_shot_power             18944 non-null  float64
 10  shooting                     18944 non-null  float64
 11  power_long_shots             18944 non-null  float64
 12  physic                       18944 non-null  float64
 13  age             

ENGINEERING THE FEATURES

In [15]:
skill_feats = ['passing','dribbling']
df['skills'] = df[skill_feats].mean(axis=1)
df.drop(columns=skill_feats, inplace=True)

shooting_feats = ['power_shot_power','shooting','power_long_shots']
df['shotlevel'] = df[shooting_feats].mean(axis=1)
df.drop(columns=shooting_feats, inplace=True)

passing_feats=['attacking_short_passing','skill_long_passing','skill_ball_control',
'skill_fk_accuracy']
df['passing']=df[passing_feats].mean(axis=1)
df.drop(columns=passing_feats,inplace=True)


movement_feats=['skill_moves','skill_dribbling','skill_curve','attacking_crossing',
'attacking_volleys','attacking_heading_accuracy','attacking_finishing']
df['movements']=df[movement_feats].mean(axis=1)
df.drop(columns=movement_feats,inplace=True)

mentality_feats=['mentality_composure','mentality_vision']
df['mentality']=df[mentality_feats].mean(axis=1)
df.drop(columns=mentality_feats,inplace=True)

power_feats=['power_strength','power_stamina']
df['power']=df[power_feats].mean(axis=1)
df.drop(columns=power_feats,inplace=True)

positioning_feats=['mentality_positioning',
'mentality_penalties']
df['power']=df[positioning_feats].mean(axis=1)
df.drop(columns=positioning_feats,inplace=True)



In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  float64
 1   potential                    18944 non-null  float64
 2   release_clause_eur           18944 non-null  float64
 3   wage_eur                     18944 non-null  float64
 4   value_eur                    18944 non-null  float64
 5   physic                       18944 non-null  float64
 6   age                          18944 non-null  float64
 7   international_reputation     18944 non-null  float64
 8   movement_reactions           18944 non-null  float64
 9   defending                    18944 non-null  float64
 10  defending_marking_awareness  18944 non-null  float64
 11  skills                       18944 non-null  float64
 12  shotlevel                    18944 non-null  float64
 13  passing         

In [47]:
features_correlations = df.corrwith(df['overall']).sort_values(ascending=False)
features_correlations

overall                        1.000000
movement_reactions             0.867232
mentality                      0.658147
skills                         0.657512
potential                      0.636358
release_clause_eur             0.614258
wage_eur                       0.588867
value_eur                      0.555109
shotlevel                      0.504925
physic                         0.493553
passing                        0.492889
age                            0.468198
international_reputation       0.440919
movements                      0.428057
power                          0.359592
defending                      0.333632
defending_marking_awareness    0.321001
dtype: float64

In [48]:
y=df['overall']
X=df.drop(['overall','international_reputation','movements','power','defending','defending_marking_awareness'],axis=1)
X

potential  release_clause_eur  wage_eur    value_eur  physic   age  \
0           93.0         138400000.0  560000.0  103500000.0    65.0  33.0   
1           92.0          75900000.0  220000.0   63000000.0    77.0  35.0   
2           91.0         132000000.0  240000.0  111000000.0    82.0  31.0   
3           91.0         166500000.0  270000.0  132000000.0    59.0  28.0   
4           91.0         161000000.0  370000.0  129000000.0    78.0  29.0   
...          ...                 ...       ...          ...     ...   ...   
18939       52.0             57000.0    1000.0      70000.0    48.0  21.0   
18940       53.0             72000.0    1000.0      70000.0    48.0  21.0   
18941       47.0             47000.0    2000.0      45000.0    59.0  28.0   
18942       67.0            165000.0     500.0     130000.0    40.0  17.0   
18943       67.0            165000.0     500.0     130000.0    45.0  17.0   

       movement_reactions  skills  shotlevel  passing  mentality  
0                    94.0    93.0  90.666667    93.00       95.5  
1                    95.0    85.0  93.333333    81.75       88.5  
2                    93.0    81.5  88.333333    81.75       83.5  
3                    91.0    90.0  83.000000    88.00       91.5  
4                    91.0    90.5  89.333333    90.50       92.5  
...                   ...     ...        ...      ...        ...  
18939                48.0    26.5  27.000000    25.50       32.5  
18940                50.0    48.0  33.000000    46.50       44.0  
18941                44.0    47.5  40.000000    47.00       39.5  
18942                53.0    46.5  45.666667    39.25       47.0  
18943                51.0    48.5  39.333333    47.25       42.5  

[18944 rows x 11 columns]

In [19]:
sc=StandardScaler()
scaled=sc.fit_transform(X)
X=pd.DataFrame(scaled, columns=X.columns)

In [20]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.1,random_state=42,stratify = y)

### USING RANDOM FOREST REGRESSOR

In [21]:
rf=RandomForestRegressor()

rf.fit(Xtrain, Ytrain)

y_pred=rf.predict(Xtest)

mean_absolute_error(y_pred,Ytest)

0.24872295514511905

### USING GRADIENT BOOSTING REGRESSOR

In [22]:
gb=GradientBoostingRegressor(init=rf,n_estimators=200,max_depth=7,learning_rate=0.085)

gb.fit(Xtrain, Ytrain)

y_pred_1 = gb.predict(Xtest)

mean_absolute_error(y_pred_1, Ytest)

0.23873755717252268

### USING XGBOOSTING REGRESSOR

In [23]:
x = XGBRegressor(n_estimators=200,max_depth=7,learning_rate=0.085)

x.fit(Xtrain, Ytrain)

y_pred_2=x.predict(Xtest)

mean_absolute_error(y_pred_2,Ytest)

0.29701711609369846

### USING VOTING REGRESSOR ENSEMBLE MODEL

In [24]:
ensemble = VotingRegressor(estimators=[('rf', rf), ('xgb', x),('gb',gb)])

ensemble.fit(Xtrain, Ytrain)

y_pred_3=ensemble.predict(Xtest)

mean_absolute_error(y_pred_3, Ytest)

0.25146785192075954

### CROSS VALIDATION

In [25]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [None, 10, 20],
    'xgb__n_estimators': [200],
    'xgb__max_depth': [7],
    'gb__n_estimators': [200],
    'gb__max_depth': [7],
    'gb__learning_rate':[0.085],
    'xgb__learning_rate':[0.085]
}

ensemble = VotingRegressor(estimators=[('rf', rf), ('xgb', x),('gb',gb)])
random_search = RandomizedSearchCV(estimator=ensemble,param_distributions=param_dist,n_iter=9,  scoring='neg_mean_squared_error',cv=5, n_jobs=-1,random_state=42
)

random_search.fit(Xtrain, Ytrain)


RandomizedSearchCV(cv=5,
                   estimator=VotingRegressor(estimators=[('rf',
                                                          RandomForestRegressor()),
                                                         ('xgb',
                                                          XGBRegressor(base_score=None,
                                                                       booster=None,
                                                                       callbacks=None,
                                                                       colsample_bylevel=None,
                                                                       colsample_bynode=None,
                                                                       colsample_bytree=None,
                                                                       device=None,
                                                                       early_stopping_rounds=None,
                                                                       enable_categorical=False,
                                                                       eval_metric=None,
                                                                       feature_types=None,
                                                                       gamma=None,
                                                                       grow_policy=Non...
                                                                                    learning_rate=0.085,
                                                                                    max_depth=7,
                                                                                    n_estimators=200))]),
                   n_iter=9, n_jobs=-1,
                   param_distributions={'gb__learning_rate': [0.085],
                                        'gb__max_depth': [7],
                                        'gb__n_estimators': [200],
                                        'rf__max_depth': [None, 10, 20],
                                        'rf__n_estimators': [100, 200, 300],
                                        'xgb__learning_rate': [0.085],
                                        'xgb__max_depth': [7],
                                        'xgb__n_estimators': [200]},
                   random_state=42, scoring='neg_mean_squared_error')

In [26]:
bm= random_search.best_estimator_

In [27]:
y_pred_4=bm.predict(Xtest)

In [28]:
mean_absolute_error(y_pred_4, Ytest)

0.25135753648642556

THE OVERALL BEST MODEL IS THE bm MODEL






### TESTING WITH FIFA22 DATASET

In [29]:
df_2=pd.read_csv('/content/drive/MyDrive/AI Projects/players_22 (1).csv',usecols=['overall', 'potential','mentality_composure', 'mentality_vision','passing', 'dribbling', 'release_clause_eur', 'wage_eur', 'value_eur',
'power_shot_power', 'shooting', 'power_long_shots','physic','age','international_reputation','attacking_short_passing', 'skill_long_passing', 'skill_ball_control', 'skill_fk_accuracy',
'skill_moves', 'skill_dribbling', 'skill_curve', 'attacking_crossing', 'attacking_volleys', 'attacking_heading_accuracy', 'attacking_finishing',
'movement_reactions','power_strength', 'power_stamina','mentality_positioning', 'mentality_penalties','defending', 'defending_marking_awareness' ])

In [30]:
df_2

overall  potential    value_eur  wage_eur  age  skill_moves  \
0           93         93   78000000.0  320000.0   34            4   
1           92         92  119500000.0  270000.0   32            4   
2           91         91   45000000.0  270000.0   36            5   
3           91         91  129000000.0  270000.0   29            5   
4           91         91  125500000.0  350000.0   30            4   
...        ...        ...          ...       ...  ...          ...   
19234       47         52      70000.0    1000.0   22            2   
19235       47         59     110000.0     500.0   19            2   
19236       47         55     100000.0     500.0   21            2   
19237       47         60     110000.0     500.0   19            2   
19238       47         60     110000.0     500.0   19            2   

       international_reputation  release_clause_eur  shooting  passing  ...  \
0                             5         144300000.0      92.0     91.0  ...   
1                             5         197200000.0      92.0     79.0  ...   
2                             5          83300000.0      94.0     80.0  ...   
3                             5         238700000.0      83.0     86.0  ...   
4                             4         232200000.0      86.0     93.0  ...   
...                         ...                 ...       ...      ...  ...   
19234                         1            114000.0      35.0     46.0  ...   
19235                         1            193000.0      39.0     50.0  ...   
19236                         1            175000.0      37.0     45.0  ...   
19237                         1            239000.0      46.0     36.0  ...   
19238                         1            217000.0      38.0     45.0  ...   

       movement_reactions  power_shot_power  power_stamina  power_strength  \
0                      94                86             72              69   
1                      93                90             76              86   
2                      94                94             77              77   
3                      89                80             81              53   
4                      91                91             89              74   
...                   ...               ...            ...             ...   
19234                  53                46             51              46   
19235                  49                49             63              47   
19236                  46                50             56              50   
19237                  48                48             50              40   
19238                  54                50             57              40   

       power_long_shots  mentality_positioning  mentality_vision  \
0                    94                     93                95   
1                    87                     95                81   
2                    93                     95                76   
3                    81                     86                90   
4                    91                     88                94   
...                 ...                    ...               ...   
19234                30                     38                43   
19235                38                     51                49   
19236                34                     45                46   
19237                41                     47                40   
19238                36                     47                46   

       mentality_penalties  mentality_composure  defending_marking_awareness  
0                       75                   96                           20  
1                       90                   88                           35  
2                       88                   95                           24  
3                       93                   93                           35  
4                       83                   89                     

In [31]:
imp = SimpleImputer(strategy='mean')
imp.fit(df_2)
imputed_data=imp.fit_transform(df_2)
df_2=pd.DataFrame(imputed_data, columns=df_2.columns)



unique_93_overall = df_2['overall'].value_counts()[93]
if unique_93_overall == 1:
    df_2['overall'].replace(93, 92, inplace=True)

In [32]:
skill_feats = ['passing','dribbling']
df_2['skills'] = df_2[skill_feats].mean(axis=1)
df_2.drop(columns=skill_feats, inplace=True)

shooting_feats = ['power_shot_power','shooting','power_long_shots']
df_2['shotlevel'] = df_2[shooting_feats].mean(axis=1)
df_2.drop(columns=shooting_feats, inplace=True)

passing_feats=['attacking_short_passing','skill_long_passing','skill_ball_control',
'skill_fk_accuracy']
df_2['passing']=df_2[passing_feats].mean(axis=1)
df_2.drop(columns=passing_feats,inplace=True)


movement_feats=['skill_moves','skill_dribbling','skill_curve','attacking_crossing',
'attacking_volleys','attacking_heading_accuracy','attacking_finishing']
df_2['movements']=df_2[movement_feats].mean(axis=1)
df_2.drop(columns=movement_feats,inplace=True)

mentality_feats=['mentality_composure','mentality_vision']
df_2['mentality']=df_2[mentality_feats].mean(axis=1)
df_2.drop(columns=mentality_feats,inplace=True)

power_feats=['power_strength','power_stamina']
df_2['power']=df_2[power_feats].mean(axis=1)
df_2.drop(columns=power_feats,inplace=True)

positioning_feats=['mentality_positioning',
'mentality_penalties']
df_2['power']=df_2[positioning_feats].mean(axis=1)
df_2.drop(columns=positioning_feats,inplace=True)


In [33]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  float64
 1   potential                    19239 non-null  float64
 2   value_eur                    19239 non-null  float64
 3   wage_eur                     19239 non-null  float64
 4   age                          19239 non-null  float64
 5   international_reputation     19239 non-null  float64
 6   release_clause_eur           19239 non-null  float64
 7   defending                    19239 non-null  float64
 8   physic                       19239 non-null  float64
 9   movement_reactions           19239 non-null  float64
 10  defending_marking_awareness  19239 non-null  float64
 11  skills                       19239 non-null  float64
 12  shotlevel                    19239 non-null  float64
 13  passing         

In [34]:
df_2

overall  potential    value_eur  wage_eur   age  \
0         92.0       93.0   78000000.0  320000.0  34.0   
1         92.0       92.0  119500000.0  270000.0  32.0   
2         91.0       91.0   45000000.0  270000.0  36.0   
3         91.0       91.0  129000000.0  270000.0  29.0   
4         91.0       91.0  125500000.0  350000.0  30.0   
...        ...        ...          ...       ...   ...   
19234     47.0       52.0      70000.0    1000.0  22.0   
19235     47.0       59.0     110000.0     500.0  19.0   
19236     47.0       55.0     100000.0     500.0  21.0   
19237     47.0       60.0     110000.0     500.0  19.0   
19238     47.0       60.0     110000.0     500.0  19.0   

       international_reputation  release_clause_eur  defending  physic  \
0                           5.0         144300000.0       34.0    65.0   
1                           5.0         197200000.0       44.0    82.0   
2                           5.0          83300000.0       34.0    75.0   
3                           5.0         238700000.0       37.0    63.0   
4                           4.0         232200000.0       64.0    78.0   
...                         ...                 ...        ...     ...   
19234                       1.0            114000.0       42.0    49.0   
19235                       1.0            193000.0       41.0    51.0   
19236                       1.0            175000.0       41.0    52.0   
19237                       1.0            239000.0       15.0    42.0   
19238                       1.0            217000.0       36.0    48.0   

       movement_reactions  defending_marking_awareness  skills  shotlevel  \
0                    94.0                         20.0    93.0  90.666667   
1                    93.0                         35.0    82.5  89.666667   
2                    94.0                         24.0    84.0  93.666667   
3                    89.0                         35.0    90.0  81.333333   
4                    91.0                         68.0    90.5  89.333333   
...                   ...                          ...     ...        ...   
19234                53.0                         38.0    47.0  37.000000   
19235                49.0                         37.0    48.0  42.000000   
19236                46.0                         38.0    47.0  40.333333   
19237                48.0                         10.0    42.0  45.000000   
19238                54.0                         33.0    46.5  41.333333   

       passing  movements  mentality  power  
0        93.00  75.857143       95.5   84.0  
1        82.00  73.285714       84.5   92.5  
2        82.25  76.000000       85.5   91.5  
3        87.25  72.142857       91.5   89.5  
4        90.25  70.000000       91.5   85.5  
...        ...        ...        ...    ...  
19234    46.25  33.714286       40.0   40.0  
19235    43.50  37.285714       48.0   47.5  
19236    46.00  33.857143       41.0   42.0  
19237    37.50  33.142857       43.5   48.0  
19238    42.25  34.571429       41.0   41.0  

[19239 rows x 17 columns]

In [44]:
y=df_2['overall']
X=df_2.drop(['overall','international_reputation','movements','power','defending','defending_marking_awareness'],axis=1)
order = ['potential','release_clause_eur','wage_eur','value_eur' ,'physic', 'age',  'movement_reactions',   'skills', 'shotlevel', 'passing',  'mentality']
X=X[order]

In [45]:
X

potential  release_clause_eur  wage_eur    value_eur  physic   age  \
0           93.0         144300000.0  320000.0   78000000.0    65.0  34.0   
1           92.0         197200000.0  270000.0  119500000.0    82.0  32.0   
2           91.0          83300000.0  270000.0   45000000.0    75.0  36.0   
3           91.0         238700000.0  270000.0  129000000.0    63.0  29.0   
4           91.0         232200000.0  350000.0  125500000.0    78.0  30.0   
...          ...                 ...       ...          ...     ...   ...   
19234       52.0            114000.0    1000.0      70000.0    49.0  22.0   
19235       59.0            193000.0     500.0     110000.0    51.0  19.0   
19236       55.0            175000.0     500.0     100000.0    52.0  21.0   
19237       60.0            239000.0     500.0     110000.0    42.0  19.0   
19238       60.0            217000.0     500.0     110000.0    48.0  19.0   

       movement_reactions  skills  shotlevel  passing  mentality  
0                    94.0    93.0  90.666667    93.00       95.5  
1                    93.0    82.5  89.666667    82.00       84.5  
2                    94.0    84.0  93.666667    82.25       85.5  
3                    89.0    90.0  81.333333    87.25       91.5  
4                    91.0    90.5  89.333333    90.25       91.5  
...                   ...     ...        ...      ...        ...  
19234                53.0    47.0  37.000000    46.25       40.0  
19235                49.0    48.0  42.000000    43.50       48.0  
19236                46.0    47.0  40.333333    46.00       41.0  
19237                48.0    42.0  45.000000    37.50       43.5  
19238                54.0    46.5  41.333333    42.25       41.0  

[19239 rows x 11 columns]

In [36]:
sc=StandardScaler()
scaled=sc.fit_transform(X)
X=pd.DataFrame(scaled, columns=X.columns)
X

potential  release_clause_eur   wage_eur  value_eur    physic  \
0       3.601780            9.591752  15.998022   9.889601  0.019139   
1       3.437470           13.244084  13.425844  15.350958  1.860333   
2       3.273160            5.380179  13.425844   5.546836  1.102194   
3       3.273160           16.109335  13.425844  16.601147 -0.197472   
4       3.273160           15.660560  17.541329  16.140551  1.427111   
...          ...                 ...        ...        ...       ...   
19234  -3.134932           -0.363165  -0.412474  -0.365905 -1.713750   
19235  -1.984762           -0.357711  -0.438196  -0.360641 -1.497139   
19236  -2.642002           -0.358953  -0.438196  -0.361957 -1.388833   
19237  -1.820452           -0.354535  -0.438196  -0.360641 -2.471889   
19238  -1.820452           -0.356054  -0.438196  -0.360641 -1.822055   

            age  movement_reactions    skills  shotlevel   passing  mentality  
0      1.851089            3.599846  3.702014   2.725584  2.731445   3.341111  
1      1.429869            3.489252  2.526349   2.654627  1.977180   2.411916  
2      2.272309            3.599846  2.694301   2.938455  1.994323   2.496389  
3      0.798039            3.046874  3.366110   2.063318  2.337170   3.003222  
4      1.008649            3.268063  3.422094   2.630975  2.542879   3.003222  
...         ...                 ...       ...        ...       ...        ...  
19234 -0.676231           -0.934526 -1.448516  -1.082447 -0.474180  -1.347096  
19235 -1.308061           -1.376903 -1.336548  -0.727662 -0.662746  -0.671319  
19236 -0.886841           -1.708687 -1.448516  -0.845924 -0.491322  -1.262624  
19237 -1.308061           -1.487498 -2.008356  -0.514791 -1.074163  -1.051444  
19238 -1.308061           -0.823931 -1.504500  -0.774967 -0.748458  -1.262624  

[19239 rows x 11 columns]

In [37]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.1,random_state=42,stratify = y)

In [38]:
y_pred_5=bm.predict(Xtest)

In [39]:
mean_absolute_error(y_pred_5, Ytest)

0.6268624590635674

### STORING THE MODEL IN A FILE

In [40]:
import pickle
filename = '/content/drive/MyDrive/AI Projects/midsem_best_model12345.pkl'
pickle.dump(bm, open(filename, 'wb'))

### DOWNLOADING THE FILE TO COMPUTER

In [42]:
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>